# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
## import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import sys, pickle, re

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection  import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score, label_ranking_average_precision_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.decomposition import TruncatedSVD

## nltk.download(['punkt','stopwords','wordnet'])

In [2]:
def load_lib(lib):
  ## Repository: C:\Users\Jeremy Sung\AppData\Roaming\nltk_data\
  ## tokenizers: 'punkt'
  ## corpora:    'stopwords', 'wordnet'
  
  try:    
    lib_name = ''
    
    if lib in ['punkt']:
      lib_name = 'tokenizers/' + lib
    elif lib in ['stopwords', 'wordnet']:
      lib_name = 'corpora/' + lib

    nltk.data.find(lib_name)
    
  except LookupError as le:
    nltk.download(lib)
    
  else:
    print('Package {} is already up-to-date!'.format(lib))

libs = ['punkt','stopwords','wordnet']
for lib in libs:
  load_lib(lib)

Package punkt is already up-to-date!
Package stopwords is already up-to-date!
Package wordnet is already up-to-date!


In [4]:
## load data from database
db_path = 'C:\\labs\\data\\disaster_messages.db'
engine = create_engine('sqlite:///' + db_path)

## Find out what tables are available in a SQLite database
## Ref. https://www.sqlitetutorial.net/sqlite-tutorial/sqlite-show-tables/

sql_show_tables = '''SELECT name
FROM sqlite_master
WHERE type = 'table' AND name NOT LIKE 'sqlite_%';
'''

df_tables = pd.read_sql(sql_show_tables, engine)
print(df_tables.name[0])

disaster_messages


In [5]:
table_name = df_tables.name[0] ## Fixed: ValueError: Table disaster_messages_tbl not found
df = pd.read_sql_table(table_name, engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
X = df['message']

## View - 
print(X[:5].values.tolist())

## result is multiple classifty.
y = df.iloc[:, 4:]
y

### 
## y = df[df.columns[5:]]
## added = pd.get_dummies(df[['related','genre']])
## y = pd.concat([y, added], axis=1)

###
## y = df.drop(['message', 'genre', 'id', 'original'], axis = 1)

['Weather update - a cold front from Cuba that could pass over Haiti', 'Is the Hurricane over or is it not over', 'Looking for someone but no name', 'UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.', 'says: west side of Haiti, rest of the country today and tonight']


,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [12]:
category_names = y.columns
## category_names
## category_names.dtype  ## dtype('O')
print(category_names.values)

['related' 'request' 'offer' 'aid_related' 'medical_help'
 'medical_products' 'search_and_rescue' 'security' 'military'
 'child_alone' 'water' 'food' 'shelter' 'clothing' 'money'
 'missing_people' 'refugees' 'death' 'other_aid' 'infrastructure_related'
 'transport' 'buildings' 'electricity' 'tools' 'hospitals' 'shops'
 'aid_centers' 'other_infrastructure' 'weather_related' 'floods' 'storm'
 'fire' 'earthquake' 'cold' 'other_weather' 'direct_report']


### 2. Write a tokenization function to process your text data

In [ ]:
## Write a tokenization function to process your text data
def tokenize(text):
  
  tokens = word_tokenize(text)
  lemmatizer = WordNetLemmatizer()
  
  clean_tokens = []
  for tok in tokens:
    clean_tok = lemmatizer.lemmatize(tok).lower().strip()
    clean_tokens.append(clean_tok)
    
  return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [14]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

In [16]:
pipeline.get_params()
## pipeline.get_params().keys()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x000001BE85B3D598>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_depth=None,
                                                          max_fea

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.2,
                                                    random_state = 45)
# train classifier
pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
def perf_report(model, X_test, y_test):
  '''
  Function to generate classification report on the model
  Input: Model, test set ie X_test & y_test
  Output: Prints the Classification report
  '''
  y_pred = model.predict(X_test)
  for i, col in enumerate(y_test):
    print(col)
    print(classification_report(y_test[col], y_pred[:, i]))

In [ ]:
perf_report(pipeline, X_test, y_test)

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters =  {'tfidf__use_idf': (True, False), 
              'clf__estimator__n_estimators': [50, 100], 
              'clf__estimator__min_samples_split': [2, 4]} 

cv = GridSearchCV(pipeline, param_grid=parameters)

In [ ]:
cv

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
## Test your model - Take a long while......
cv.fit(X_train, y_train)
perf_report(cv, X_test, y_test)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [17]:
#Improve  the pipeline
pipeline2 = Pipeline([
    ('vect', CountVectorizer()),
    ('best', TruncatedSVD()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [18]:
pipeline2.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'best', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'best__algorithm', 'best__n_components', 'best__n_iter', 'best__random_state', 'best__tol', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__algorithm', 'clf__estimator__base_estimator', 'clf__estimator__learning_rate', 'clf__estimator__n_estimators', 'clf__estimator__random_state', 'clf__estimator', 'clf__n_jobs'])

In [ ]:
#Train & predict
pipeline2.fit(X_train, y_train)
perf_report(pipeline2, X_test, y_test)

In [ ]:
#Param tunning 
parameters2 = { #'vect__ngram_range': ((1, 1), (1, 2)), 
              #'vect__max_df': (0.5, 1.0), 
              #'vect__max_features': (None, 5000), 
              'tfidf__use_idf': (True, False), 
              'clf__estimator__n_estimators': [50, 100],
              'clf__estimator__learning_rate': [1,2] }

In [ ]:
cv2 = GridSearchCV(pipeline2, param_grid=parameters2)
cv2

In [ ]:
cv2.fit(X_train, y_train)

In [ ]:
perf_report(cv2, X_test, y_test)

### 9. Export your model as a pickle file

In [ ]:
with open('c:/labs/data/model.pkl', 'wb') as f:
  pickle.dump(cv2, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [19]:
## def find_tables(db_path, engine):
def find_tables(engine):
  ## Find out what tables are available in a SQLite database
  ## Ref. https://www.sqlitetutorial.net/sqlite-tutorial/sqlite-show-tables/
  
  sql_show_tables = '''
  SELECT name
  FROM sqlite_master
  WHERE type = 'table' AND name NOT LIKE 'sqlite_%';
  '''

  df_tables = pd.read_sql(sql_show_tables, engine)
  
  return df_tables.name[0]


def load_data(db_path, table_name, engine):
  
  if not table_name:
    table_name = find_tables(db_path, engine)
    
  ## df = pd.read_sql_table(table_name, engine)  
  return pd.read_sql_table(table_name, engine)


def tokenize(text):
  ## Tokenization function to process your text data
  
  tokens = word_tokenize(text)
  lemmatizer = WordNetLemmatizer()
  
  clean_tokens = []
  for tok in tokens:
    clean_tok = lemmatizer.lemmatize(tok).lower().strip()
    clean_tokens.append(clean_tok)
    
  return clean_tokens


def perf_report(model, X_test, y_test):
  '''
  Function to generate classification report on the model
  Input: Model, test set ie X_test & y_test
  Output: Prints the Classification report
  '''
  y_pred = model.predict(X_test)
  for i, col in enumerate(y_test):
    print(col)
    print(classification_report(y_test[col], y_pred[:, i]))


def train_pipeline(pipeline, X_train, y_train):

  pipeline.get_params()

  ## train classifier
  pipeline.fit(X_train, y_train)
  
  return pipeline


def optimize_model(pipeline, X_train, y_train, parameters):  
  ## Improve your model
  #   parameters =  {'tfidf__use_idf': (True, False), 
  #                 'clf__estimator__n_estimators': [50, 100], 
  #                 'clf__estimator__min_samples_split': [2, 4]} 

  optimized_model = GridSearchCV(pipeline, param_grid= parameters)

  ## Test your model - Take a long while......
  optimized_model.fit(X_train, y_train)
  
  return optimized_model
  

def evaluate_model(y_test, y_pred, category_names):
  result = precision_recall_fscore_support(y_test, y_pred)
  for i, col in enumerate(category_names):
    accu = accuracy_score(y_test.loc[:,col],y_pred[:,i])
    print('{}\n Accuracy: {:.4f}% Precision: {:.4f}% Recall {:.4f} '.format(
      col, accu, result[0][i], result[1][i]))


def export_model(model, export_path='c:/labs/data/default_path_model.pkl'):
  ## Export your model as a pickle file

  with open(export_path, 'wb') as f:
    pickle.dump(model, f)



def main():

  print("Load data from database")
  db_path = 'C:\\labs\\data\\disaster_messages.db'
  engine = create_engine('sqlite:///' + db_path)

  print("Getting table name...")
  table_name = find_tables(engine)
  
  print("Loading data to DataFrame...")
  df = load_data(db_path, table_name, engine)

  print("Data Loaded: {}".format(df.head()))  
  
  X = df['message']
  y = df.iloc[:, 4:]
  category_names = y.columns.values

  X_train, X_test, y_train, y_test = train_test_split(X,
                                                      y,
                                                      test_size = 0.2,
                                                      random_state = 45)

  print("Build a machine learning pipeline")
  pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer= tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

  print("Train pipeline...") 
  model = train_pipeline(pipeline, X_train, y_train)
  
  perf_report(model, X_test, y_test)
  
  print("Improve Model")
  parameters =  {'tfidf__use_idf': (True, False), 
              'clf__estimator__n_estimators': [50, 100], 
              'clf__estimator__min_samples_split': [2, 4]} 

  optimized_model = optimize_model(model, X_train, y_train, parameters)
  
  perf_report(optimized_model, X_test, y_test)
  
  print("Evaluating model")
  y_pred = optimized_model.predict(X_test)
  evaluate_model(y_test, y_pred, category_names)
  
  print("Export model...")
  export_path = 'c:/labs/data/model.pkl'
  export_model(optimized_model, export_path)
  
  
  print("Build another pipeline...") 
  pipeline2 = Pipeline([
      ('vect', CountVectorizer()),
      ('best', TruncatedSVD()),
      ('tfidf', TfidfTransformer()),
      ('clf', MultiOutputClassifier(AdaBoostClassifier()))
  ])
  
  #Train & predict
  model2 = train_pipeline(pipeline2, X_train, y_train)
   
  perf_report(model2, X_test, y_test)

  print("Parameter tunning") 
  parameters2 = {'tfidf__use_idf': (True, False), 
                'clf__estimator__n_estimators': [50, 100],
                'clf__estimator__learning_rate': [1,2] }

  optimized_model2 = optimize_model(model2, X_train, y_train, parameters2)

  perf_report(optimized_model2, X_test, y_test)

  print("Evaluating model")
  y_pred2 = optimized_model2.predict(X_test)
  evaluate_model(y_test, y_pred2, category_names)
  
  print("Export model2...")
  export_path2 = 'c:/labs/data/model2.pkl'
  export_model(optimized_model2, export_path2)

if __name__ == "__main__":
  main()

Load data from database
Getting table name...
Loading data to DataFrame...
Data Loaded:    id                                            message  \
0   2  Weather update - a cold front from Cuba that c...   
1   7            Is the Hurricane over or is it not over   
2   8                    Looking for someone but no name   
3   9  UN reports Leogane 80-90 destroyed. Only Hospi...   
4  12  says: west side of Haiti, rest of the country ...   

                                            original   genre  related  \
0  Un front froid se retrouve sur Cuba ce matin. ...  direct        1   
1                 Cyclone nan fini osinon li pa fini  direct        1   
2  Patnm, di Maryani relem pou li banm nouvel li ...  direct        1   
3  UN reports Leogane 80-90 destroyed. Only Hospi...  direct        1   
4  facade ouest d Haiti et le reste du pays aujou...  direct        1   

   request  offer  aid_related  medical_help  medical_products  ...  \
0        0      0            0           

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: F

related
              precision    recall  f1-score   support

           0       0.62      0.36      0.45      1222
           1       0.83      0.93      0.88      4021

    accuracy                           0.80      5243
   macro avg       0.72      0.65      0.67      5243
weighted avg       0.78      0.80      0.78      5243

request
              precision    recall  f1-score   support

           0       0.88      0.98      0.93      4345
           1       0.82      0.35      0.49       898

    accuracy                           0.88      5243
   macro avg       0.85      0.67      0.71      5243
weighted avg       0.87      0.88      0.85      5243

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5216
           1       0.00      0.00      0.00        27

    accuracy                           0.99      5243
   macro avg       0.50      0.50      0.50      5243
weighted avg       0.99      0.99      0.99      524

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.95      1.00      0.97      4917
           1       0.82      0.17      0.28       326

    accuracy                           0.95      5243
   macro avg       0.89      0.58      0.63      5243
weighted avg       0.94      0.95      0.93      5243

food
              precision    recall  f1-score   support

           0       0.92      0.99      0.95      4648
           1       0.79      0.30      0.43       595

    accuracy                           0.91      5243
   macro avg       0.85      0.64      0.69      5243
weighted avg       0.90      0.91      0.89      5243

shelter
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      4808
           1       0.84      0.17      0.29       435

    accuracy                           0.93      5243
   macro avg       0.89      0.58      0.62      5243
weighted avg       0.92      0.93      0.91      5243

clothi

...

related
              precision    recall  f1-score   support

           0       0.50      0.00      0.01      1222
           1       0.77      1.00      0.87      4021

    accuracy                           0.77      5243
   macro avg       0.63      0.50      0.44      5243
weighted avg       0.71      0.77      0.67      5243

request
              precision    recall  f1-score   support

           0       0.83      1.00      0.91      4345
           1       0.00      0.00      0.00       898

    accuracy                           0.83      5243
   macro avg       0.41      0.50      0.45      5243
weighted avg       0.69      0.83      0.75      5243

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5216
           1       0.00      0.00      0.00        27

    accuracy                           0.99      5243
   macro avg       0.50      0.50      0.50      5243
weighted avg       0.99      0.99      0.99      524

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.94      1.00      0.97      4917
           1       0.00      0.00      0.00       326

    accuracy                           0.94      5243
   macro avg       0.47      0.50      0.48      5243
weighted avg       0.88      0.94      0.91      5243

food
              precision    recall  f1-score   support

           0       0.89      1.00      0.94      4648
           1       0.00      0.00      0.00       595

    accuracy                           0.89      5243
   macro avg       0.44      0.50      0.47      5243
weighted avg       0.79      0.89      0.83      5243

shelter
              precision    recall  f1-score   support

           0       0.92      1.00      0.96      4808
           1       0.00      0.00      0.00       435

    accuracy                           0.92      5243
   macro avg       0.46      0.50      0.48      5243
weighted avg       0.84      0.92      0.88      5243

clothi

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


related
 Accuracy: 0.7669% Precision: 0.7675% Recall 0.9985 
request
 Accuracy: 0.8285% Precision: 0.0000% Recall 0.0000 
offer
 Accuracy: 0.9949% Precision: 0.0000% Recall 0.0000 
aid_related
 Accuracy: 0.5894% Precision: 0.5000% Recall 0.0084 
medical_help
 Accuracy: 0.9250% Precision: 0.0000% Recall 0.0000 
medical_products
 Accuracy: 0.9521% Precision: 0.0000% Recall 0.0000 
search_and_rescue
 Accuracy: 0.9699% Precision: 0.0000% Recall 0.0000 
security
 Accuracy: 0.9792% Precision: 0.0000% Recall 0.0000 
military
 Accuracy: 0.9649% Precision: 0.0000% Recall 0.0000 
child_alone
 Accuracy: 1.0000% Precision: 0.0000% Recall 0.0000 
water
 Accuracy: 0.9378% Precision: 0.0000% Recall 0.0000 
food
 Accuracy: 0.8861% Precision: 0.0000% Recall 0.0000 
shelter
 Accuracy: 0.9170% Precision: 0.0000% Recall 0.0000 
clothing
 Accuracy: 0.9889% Precision: 0.0000% Recall 0.0000 
money
 Accuracy: 0.9758% Precision: 0.0000% Recall 0.0000 
missing_people
 Accuracy: 0.9870% Precision: 0.0000% Recall